In [1]:
import os
os.getcwd()

'c:\\Mohit\\Imperial\\fyp-hearts\\notebooks\\data_exploration'

In [2]:
os.chdir("../..")
os.getcwd()

'c:\\Mohit\\Imperial\\fyp-hearts'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import utils

In [5]:
datafolder = os.getcwd()+'/datasets/PTB-XL/'
sampling_frequency = 100
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)

In [6]:
subset=['age', 'weight', 'sex', 'height']
labels, data = utils.select_subset(raw_labels, data, subset=subset)

In [18]:
raw_labels.columns

Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')

In [8]:
labels1 = utils.compute_label_aggregations(labels, datafolder, "exp0")

c:\Mohit\Imperial\fyp-hearts\utils.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['scp_codes_len'] = df.scp_codes.apply(lambda x: len(x))


In [14]:
outputfolder = ''
data, labels1, Y1, _ = utils.select_data(data, labels, "all", 0, os.getcwd())

AttributeError: 'DataFrame' object has no attribute 'all_scp'

In [19]:
class DEMO_Experiment:
    """
    Run standard experiments on ECG datasets
    """
    
    def __init__(self, experiment_name, task, datafolder, outputfolder, models, sampling_frequency=100, min_samples=0, train_fold=8, val_fold=9, test_fold=10, folds_type='strat'):
        """
        Parameters:
        experiment_name : str
            The experiment name, used for saving outputs as well.
        task : 
        """
        self.experiment_name = experiment_name
        self.task = task
        self.datafolder = datafolder
        self.outputfolder = outputfolder
        self.models = models
        self.sampling_frequency = sampling_frequency
        self.min_samples = min_samples
        self.train_fold = train_fold
        self.val_fold = val_fold
        self.test_fold = test_fold
        self.folds_type = folds_type
        
        print(self.models)
        
        # if needed create folder structure
        if not os.path.exists(self.outputfolder+self.experiment_name):
            os.makedirs(self.outputfolder+self.experiment_name)
        if not os.path.exists(self.outputfolder+self.experiment_name+'/results/'):
            os.makedirs(self.outputfolder+self.experiment_name+'/results/')
        if not os.path.exists(self.outputfolder+self.experiment_name+'/models/'):
            os.makedirs(self.outputfolder+self.experiment_name+'/models/')
        if not os.path.exists(self.outputfolder+self.experiment_name+'/data/'):
            os.makedirs(self.outputfolder+self.experiment_name+'/data/')
        
        print(self.outputfolder+self.experiment_name+'/data/')
        
    def prepare(self):
        # Load PTB-XL data
        self.data, self.raw_labels = utils.load_dataset(self.datafolder, self.sampling_frequency)
        
        # Preprocess label data
        subset=['age', 'weight', 'sex', 'height']
        # Get data only where demographic data is available
        self.labels, self.data = utils.select_subset(self.raw_labels, self.data, subset=subset)
        self.demographics = self.labels[subset]
        self.labels = utils.compute_label_aggregations(self.labels, self.datafolder, self.task)
        
        # One hot encode relevant data
        self.data, self.labels, self.Y, _ = utils.select_data(self.data, self.labels, self.task, self.min_samples, self.outputfolder+self.experiment_name+'/data/')
        return self.data, self.labels, self.Y, self.raw_labels

In [20]:
from configs.demo_wavelet_configs import *
outputfolder = os.getcwd()+'/output/'
datafolder = os.getcwd()+'/datasets/PTB-XL/'
models = [conf_wavelet_demo_standard_rf]
experiments = [('exp0', 'all')]
for name, task in experiments:
        f = DEMO_Experiment(name, task, datafolder, outputfolder, models)
        data, labels, y, raw_labels = f.prepare()
        

[{'modelname': 'Wavelet+demo+RF', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'RF'}}]
c:\Mohit\Imperial\fyp-hearts/output/exp0/data/


c:\Mohit\Imperial\fyp-hearts\utils.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['scp_codes_len'] = df.scp_codes.apply(lambda x: len(x))
c:\Mohit\Imperial\fyp-hearts\utils.py:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['all_scp'] = df.scp_codes.apply(lambda x: list(set(x.keys())))
c:\Mohit\Imperial\fyp-hearts\utils.py:302: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [33]:
len(np.unique(y[labels['strat_fold']==10][:,0]))

2

In [39]:
def exp_check(exp, y, labels, dic):
    cols = y.shape[1]
    print(exp)
    vals = []
    for col in range(cols):
        vals += [len(np.unique(y[labels['strat_fold']==10][:,col]))]
    
    vals = np.array(vals)
    print(vals)
    dic[exp] = vals 
    return dic


experiments = [
        ('exp0', 'all'),
        ('exp1', 'diagnostic'),
        ('exp1.1', 'subdiagnostic'),
        ('exp1.1.1', 'superdiagnostic'),
        ('exp2', 'form'),
        ('exp3', 'rhythm')
    ]

dic_exps = {}
for name, task in experiments:
        f = DEMO_Experiment(name, task, datafolder, outputfolder, models)
        data, labels, y, raw_labels = f.prepare()
        dic_exps = exp_check(task, y, labels, dic_exps)

[{'modelname': 'Wavelet+demo+RF', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'RF'}}]
c:\Mohit\Imperial\fyp-hearts/output/exp0/data/


c:\Mohit\Imperial\fyp-hearts\utils.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['scp_codes_len'] = df.scp_codes.apply(lambda x: len(x))
c:\Mohit\Imperial\fyp-hearts\utils.py:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['all_scp'] = df.scp_codes.apply(lambda x: list(set(x.keys())))
c:\Mohit\Imperial\fyp-hearts\utils.py:302: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

all
[2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[{'modelname': 'Wavelet+demo+RF', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'RF'}}]
c:\Mohit\Imperial\fyp-hearts/output/exp1/data/


c:\Mohit\Imperial\fyp-hearts\utils.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['scp_codes_len'] = df.scp_codes.apply(lambda x: len(x))
c:\Mohit\Imperial\fyp-hearts\utils.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diagnostic'] = df.scp_codes.apply(aggregate_all_diagnostic)
c:\Mohit\Imperial\fyp-hearts\utils.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

diagnostic
[2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2]
[{'modelname': 'Wavelet+demo+RF', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'RF'}}]
c:\Mohit\Imperial\fyp-hearts/output/exp1.1/data/


...

superdiagnostic
[2 2 2 2 2]
[{'modelname': 'Wavelet+demo+RF', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'RF'}}]
c:\Mohit\Imperial\fyp-hearts/output/exp2/data/


c:\Mohit\Imperial\fyp-hearts\utils.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['scp_codes_len'] = df.scp_codes.apply(lambda x: len(x))
c:\Mohit\Imperial\fyp-hearts\utils.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['form'] = df.scp_codes.apply(aggregate_form)
c:\Mohit\Imperial\fyp-hearts\utils.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

form
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[{'modelname': 'Wavelet+demo+RF', 'modeltype': 'WAVELET', 'modelparams': {'regularizer_C': 0.001, 'classifier': 'RF'}}]
c:\Mohit\Imperial\fyp-hearts/output/exp3/data/


c:\Mohit\Imperial\fyp-hearts\utils.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['scp_codes_len'] = df.scp_codes.apply(lambda x: len(x))
c:\Mohit\Imperial\fyp-hearts\utils.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rhythm'] = df.scp_codes.apply(aggregate_rhythm)
c:\Mohit\Imperial\fyp-hearts\utils.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

rhythm
[2 2 2 2 2 2 2 2 2 2 2]


In [41]:
fic_e = dic_exps
for key in dic_exps:
    val = dic_exps[key]
    val = len(val[val==2])/len(val)
    dic_exps[key] = val

dic_exps

{'all': 0.9558823529411765,
 'diagnostic': 0.9318181818181818,
 'subdiagnostic': 1.0,
 'superdiagnostic': 1.0,
 'form': 1.0,
 'rhythm': 1.0}